<a href="https://colab.research.google.com/github/alexandriaorvis/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = ["T9", "T13", "T12","T2","T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_keep = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']
classifications_to_replace = []
for item in application_df['CLASSIFICATION']:
    if item not in classifications_to_keep:
      classifications_to_replace.append(item)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [23]:
# Make a copy of the dataframe as it is now for later use
application_optimized = application_df.copy()

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_convert = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']

dummy_columns = pd.get_dummies(application_df[columns_to_convert])

application_df = pd.concat([application_df, dummy_columns], axis=1)

application_df = application_df.drop(columns=columns_to_convert)


In [25]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop("IS_SUCCESSFUL", axis=1)
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model Optimization 1 : Failed

In [27]:
## Change both hidden layer activation function to sigmoid

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="sigmoid", input_dim=1))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics = ['accuracy'])

In [30]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_{epoch:02d}.h5",
    save_weights_only=True,
    period=5
)

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5950 - accuracy: 0.7007
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5734 - accuracy: 0.7207
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5686 - accuracy: 0.7228
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5629 - accuracy: 0.7253
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5591 - accuracy: 0.7272
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7289
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5545 - accuracy: 0.7297
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7305
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5520 - accura

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5527 - accuracy: 0.7285 - 493ms/epoch - 2ms/step
Loss: 0.5527458190917969, Accuracy: 0.7285131216049194


## Model Optimization 2 : Failed

In [31]:
 ## Add 50 neurons to the second hidden layer and a third hidden layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=1))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu', input_dim=1))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 80)                6480      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 12461 (48.68 KB)
Trainable params: 12461 (48.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics = ['accuracy'])

In [33]:
checkpoint_callback_3 = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_{epoch:02d}.h5",
    save_weights_only=True,
    period=5
)

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 5s 3ms/step - loss: 0.5690 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5539 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5507 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5493 - accuracy: 0.7323
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5483 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7343
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5469 - accuracy: 0.7357
Epoch 8/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5461 - accuracy: 0.7348
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5455 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 15s 18ms/step - loss: 0.5457 - accu

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5742 - accuracy: 0.7284 - 492ms/epoch - 2ms/step
Loss: 0.5741783380508423, Accuracy: 0.728396475315094


## Model Optimization 3 : Failed

In [36]:
## Reprocess original data because the source df is different

In [38]:
## Drop columns to attempt to improve model function using copied dataframe from above
application_optimized = application_optimized.drop(columns=["AFFILIATION", "STATUS", "SPECIAL_CONSIDERATIONS"])
application_optimized.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,0,5000,1
1,T3,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,C3000,ProductDev,Association,0,5000,0
3,T3,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,C1000,Heathcare,Trust,100000-499999,142590,1


In [39]:
# Turn catagorical variables into dummy variables
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_convert = ['APPLICATION_TYPE', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

dummy_columns = pd.get_dummies(application_optimized[columns_to_convert])

application_optimized = pd.concat([application_optimized, dummy_columns], axis=1)

application_optimized = application_optimized.drop(columns=columns_to_convert)


In [40]:
application_optimized.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [41]:
# Split our preprocessed data into our features and target arrays
X = application_optimized.drop("IS_SUCCESSFUL", axis=1)
y = application_optimized["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
## Run the most successful model (the first) on the altered data frame

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=34))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=1))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                2800      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5261 (20.55 KB)
Trainable params: 5261 (20.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics = ['accuracy'])

In [45]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights_{epoch:02d}.h5",
    save_weights_only=True,
    period=5
)

In [46]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6224 - accuracy: 0.6442
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6124 - accuracy: 0.6530
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6096 - accuracy: 0.6544
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6087 - accuracy: 0.6545
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6072 - accuracy: 0.6560
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6073 - accuracy: 0.6554
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6069 - accuracy: 0.6550
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6065 - accuracy: 0.6577
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6051 - accuracy: 0.6576
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6055 - accura

In [47]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
